In [1]:
import os
import pygame
import time 
import sys
import pandas as pd
import numpy as np
import shutil
from matplotlib import cm
from collections import Counter
from PIL import Image, ImageOps 
import random

pygame 2.0.1 (SDL 2.0.14, Python 3.9.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


# if converting otf to ttf, remove AnyConv.com__

In [2]:
font_file = 'fonts'
image_file = 'images'

In [3]:
fonts = os.listdir(font_file)

path = "/Users/himanshu/Desktop/DL_project/datasets/fonts/"

for name in fonts:
    if "AnyConv.com__" in name:
        new_name = name.replace("AnyConv.com__", "")
        os.rename(path + name, path +new_name)
        
        print(name, "changed")


# from utils

In [4]:

import os
import pygame
import time 
import sys
import pandas as pd
import numpy as np
import shutil
from matplotlib import cm
from collections import Counter
from PIL import Image

pygame.init()


def render_text_outline(font, text, color, background, outline, outlinecolor):
    outlineSurf = font.render(text, True, outlinecolor)
    outlineSize = outlineSurf.get_size()
    textSurf = pygame.Surface((outlineSize[0] + outline*2, outlineSize[1] + 2*outline))
    textSurf.fill(background)
    textRect = textSurf.get_rect()
    offsets = [(ox, oy) 
        for ox in range(-outline, 2*outline, outline)
        for oy in range(-outline, 2*outline, outline)
        if ox != 0 or ox != 0]
    for ox, oy in offsets:   
        px, py = textRect.center
        textSurf.blit(outlineSurf, outlineSurf.get_rect(center = (px+ox, py+oy))) 
    innerText = font.render(text, True, color).convert_alpha()
    textSurf.blit(innerText, innerText.get_rect(center = textRect.center)) 
    return textSurf


def font2image(input_file, output_paths, characters, size):
    input_file_name = input_file.split(os.sep)[-1].split('.')[0]   # get output file_name
    output_path = os.path.join(output_paths, input_file_name)
    
    if not os.path.exists(output_path):
        os.mkdir(output_path)
        
    AZ = [chr(i) for i in range(0x0041,0x005A+1)]    
    file_sizes=[]
    
    for word in characters:
        window = pygame.display.set_mode((size, size))
        window.fill((255, 255, 255))
        
        font = pygame.font.Font(input_file, size)
        
        text_outline = render_text_outline(font, str(word[0]), (0, 0, 0), (255, 255, 255), 10, (255, 0, 0))
        
        if word in AZ:      # for uppercase letter
            word = word+'+'
        
        window.blit(text_outline, text_outline.get_rect(center = window.get_rect().center))
        pygame.image.save(window, os.path.join(output_path,word+".jpeg"))
        
        pygame.display.quit()
        
    remove_duplicated_images(output_path)
    process_image(output_path, size)

    
def remove_duplicated_images(path):
    while True:
        files = os.listdir(path)
        if len(files)==0:
            print('!!!!!!!!!!!!!!!!!!error:{}'.format(path))
            break
        file_sizes = []
        for file in files:
            file_size = os.path.getsize(os.path.join(path,file))
            file_sizes.append(file_size)
        counter = Counter(file_sizes)
        most_common_number = counter.most_common(1)[0][1]
        if most_common_number<=10:
            break
        most_common = counter.most_common(1)[0][0]
        for file in files:                                        # remove empty images
            file_path = os.path.join(path, file)
            if os.path.getsize(file_path)==most_common:
                os.remove(file_path)  
                
def load_image(path):
    image = Image.open(path).convert('L')
    image = np.array(image)
    return image

def cut_image(image):
    (h, w) = image.shape
    h_value = 255*h
    w_value = 255*w
    left = 0
    right = w
    upper = 0
    bottom = h
    for r in range(w):
        value = image[:, r].sum()
        if value==h_value:
            left += 1
        else:
            break
    for r in range(w-1, -1, -1):
        value = image[:,r].sum()
        if value==h_value:
            right -= 1
        else:
            break
    for c in range(h):
        value = image[c, :].sum()
        if value==w_value:
            upper += 1
        else:
            break
    for c in range(h-1, -1, -1):
        value = image[c, :].sum()
        if value==w_value:
            bottom -= 1
        else:
            break
    if left==w or right==0 or upper==h or bottom==0:
        left = 0
        right = w
        upper = 0
        bottom = h
    image_cut = image[upper:bottom, left:right]
    return image_cut

def resize_image(image_cut, size):
    (h, w) = image_cut.shape
    image_p = Image.fromarray(np.uint8(cm.gray(image_cut)*255))
    image_resized = image_p
    if h>w:
        if h>size:
            ratio = h/size
            adjust = int(w/ratio)
            if adjust<1:
                adjust=1
            image_resized = image_p.resize((adjust, size))
    else:
        if w>size:
            ratio = w/size
            adjust = int(h/ratio)
            if adjust<1:
                adjust=1
            image_resized = image_p.resize((size, adjust))
    return image_resized

def pad_image(image_resized, size):
    back = Image.new('L', (size, size), color=255)
    h_r, v_r = image_resized.size
    h = int((size-h_r)/2)
    v = int((size-v_r)/2)
    back.paste(image_resized,(h, v))
    return back

def color_text(image, color):
    '''
    By himanshu
    coloring the text
    '''
    return ImageOps.colorize(image, black =color, white ="white")

def get_random_color():
    
    color = 'rgb(' + str(random.randint(0, 255)) + ',' + str(random.randint(0, 255)) + ',' + str(random.randint(0, 255)) +')'
    
    return color

def process_image(path, size):
    files = os.listdir(path)
    
    color = get_random_color()
    
    for file in files:
        file_path = os.path.join(path, file)
        image = load_image(file_path)
        image = cut_image(image)
        image = resize_image(image, size)
        image = pad_image(image, size)
        image = color_text(image, color)
        image.save(file_path)
        
def remove_empty_floder(path):
    files = os.listdir(path)
    for file in files:
        if not os.listdir(os.path.join(path,file)):
            os.rmdir(os.path.join(path,file))
            print(file,' |removed')
    print("done!")
    
# check current font exists the given characters or not 
def check_image_exists(path, characters):
    AZ = [chr(i) for i in range(0x0041,0x005A+1)]  
    for word in characters:
        if word in AZ:      
            word = word+'+'
        image = word+'.png'
        image_path = os.path.join(path, image)
        if not os.path.exists(image_path):
            print('no ', word)
    print('done!')

In [5]:
common_list = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz1234567890"

# adding 240 japanese characters
common_list += "一二三四五六七八九十百千上下左右中大小月日年早木林山川土空田天生花草虫犬人名女男子目耳口手足見音力気円入出立休先夕本文字学校村町森正水火玉王石竹糸貝車金雨赤青白数多少万半形太細広長点丸交光角計直線矢弱強高同親母父姉兄弟妹自友体毛頭顔首心時曜朝昼夜分週春夏秋冬今新古間方北南東西遠近前後内外場地国園谷野原里市京風雪雲池海岩星室戸家寺通門道話言答声聞語読書記紙画絵図工教晴思考知才理算作元食肉馬牛魚鳥羽鳴麦米茶色黄黒来行帰歩走止活店買売午汽弓回会組船明社切電毎合当台楽公引科歌刀番用何"



In [6]:

fonts = os.listdir(font_file)

done = 0

for font in fonts:
    try:
        font_path = os.path.join(font_file, font)
        font2image(font_path, image_file, common_list, 1000)
        done += 1
        print(font, "done")
        
        if(done == 1):
            break
    except:
        print(font, "failed......")

# exiting pygame
pygame.quit()
exit()
        
print("Total fonts done = ", done)

!!!!!!!!!!!!!!!!!!error:images/Kaiso-Next-B
Kaiso-Next-B.ttf done
Total fonts done =  1
